# How to learn pressure term

In [1]:
### Learn pressure term through its gradient:
###     Setup neural network, train through gradient
###     - Function that evaluates gradient
###     - Training data: Average of (Unew + Uforcing term)/dt
###     - To lock down the constant: Need condition
###         - Sum of pressure = 0
###         - Pressure at a point = 0

In [2]:
import time
import numpy as np
import math

import torch
import torch.nn as nn
import torch.optim as optim

import DRLPDE_functions.EvaluateWalkers
 
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300


In [3]:
global L_height, v0

############## Save model and/or Load model ##############

# Physical Dimension
x_dim = 2
output_dim = 2

is_unsteady = False
input_dim = x_dim + is_unsteady

L_height = 0.5
v0 = 1.513787

# Is there a true solution
exists_analytic_sol = False
# If there is a true solution, provide contour levels
plot_levels = np.linspace(-1,1,100)

def true_solution(X):
    pass

################# PDE Coefficients ########################

# PDE type:
pde_type = 'NavierStokes'

# Diffusion coefficient
mu = 1

# Forcing term
def forcing(X):
    f = torch.zeros( (X.size(0), output_dim), device=X.device)
    return f

################# Boundary and Initial Conditions ###########
# Use pytorch expressions to make boundary and initial conditions 
#
# To make different boundary conditions for each boundary
#     ensure the correct bdry_con is called when defining the boundaries

def bdry_con(X):
    u = torch.zeros( (X.size(0), output_dim), device=X.device)
    return u

def inlet_con(X):
    u = torch.zeros_like(X, device=X.device)
    
    u[:,0] = v0*torch.mul((L_height - X[:,1]),(L_height + X[:,1]))/(L_height**2)

    return u

#################  Make the domain  #######################
#     First define a bounding box containing your domain
#         Syntax: [ x interval, y interval, z interval ]
#         Points will be sampled through rejection sampling
#
#     Define each boundary
#         lines: [ 'line', point, normal, endpoints, bdry_condition ]
#         disk:  [ 'disk', centre, radius, endpoints, bdry_condition]
#         
#     Intersections of boundaries must be input manually
#         These should be ordered as points will be sampled from first to second
#         only 2 intersection points allowed
#         
#     Boundary condition is given by a function using pytorch expressions


boundingbox = [ [0, 5*L_height], [-L_height,L_height] ]

wall_left = {'type':'line',
             'point': [0, -L_height],
             'normal': [1,0],
             'endpoints': [ [0, -L_height], [0, L_height] ],
             'boundary_condition': inlet_con }

wall_top = { 'type':'line',
             'point': [0, L_height],
             'normal':  [0,-1],
             'endpoints': [ [0, L_height], [5*L_height, L_height] ],
             'boundary_condition': bdry_con }

wall_bot = {'type':'line',
             'point': [0,-L_height],
             'normal': [0, 1],
             'endpoints': [ [0, -L_height], [5*L_height, -L_height] ],
             'boundary_condition': bdry_con }

list_of_dirichlet_boundaries = [wall_left, wall_top, wall_bot ]
list_of_periodic_boundaries =[]

In [4]:
# Time step
dt = 1e-2
# exit tolerance
tol = 1e-6

# Number of walkers

numpts = 2**12

#numpts_x = 2**8
#numpts_y = 2**4

num_ghost = 512
num_batch = 2**8

# Update walkers
# Options: 
#    move -- moves walkers to one of their new locations
#    remake -- remake walkers at each training step
#    fixed -- keeps walkers fixed
update_walkers = 'fixed'
update_walkers_every = 1
############## Training Parameters #######################

# Training epochs
num_epoch = 5000

update_print_every = 1000

# Neural Network Architecture
nn_depth = 20
nn_width = 4

# Weighting of losses
lambda_bell = 1e2
lambda_fix = 1e0

# Learning rate
learning_rate = 1e-2
adam_beta = (0.9,0.999)
weight_decay = 0

# Fixed pressure
fixed_pressure  = 80
fix_pressure_at = [dt,0]

In [5]:
class FeedForwardNN(nn.Module):
    
    ### Feed forward neural network
    
    def __init__(self, depth, width, x_dim, is_unsteady, output_dim, **nn_param):
        super(FeedForwardNN, self).__init__()
        
        self.input_dim = x_dim + is_unsteady
        
        modules = []
        modules.append(nn.Linear(self.input_dim, depth))
        for i in range(width - 1):
            modules.append(nn.Linear(depth, depth))
            modules.append(nn.ELU())
        modules.append(nn.Linear(depth, output_dim))
        
        self.sequential_model = nn.Sequential(*modules)
        
    def forward(self, x):
        a = self.sequential_model(x)
        
        return a

class PotentialNN(nn.Module):
    
    def __init__(self, depth, width, x_dim, is_unsteady, output_dim, **nn_param):
        super(PotentialNN, self).__init__()
        
        self.input_dim = x_dim + is_unsteady
        
        modules = []
        modules.append(nn.Linear(self.input_dim, depth))
        for i in range(width - 1):
            modules.append(nn.Linear(depth, depth))
            modules.append(nn.Tanh())
        modules.append(nn.Linear(depth, 1))
        
        self.sequential_model = nn.Sequential(*modules)
        
    def forward(self, x):
        p = self.sequential_model(x)
        
        a = torch.autograd.grad(p, x, grad_outputs = torch.ones_like(p), create_graph = True, 
                                        retain_graph = True, only_inputs = True)[0]

        return a

    def evaluate_potential(self,x):
        p = self.sequential_model(x)

        return p

In [6]:
class Domain:
    ### This class defines the domain using the parameters provided
    ### It sets up the boundingbox and each boundary

    def __init__(self, is_unsteady, boundingbox, 
                list_of_dirichlet_boundaries,
                list_of_periodic_boundaries):
        
        self.boundingbox = boundingbox
        self.is_unsteady = is_unsteady

        self.num_of_boundaries = len(list_of_dirichlet_boundaries)
        
        # Unpack dirichlet boundary descriptions
        self.boundaries = []
        for specs in list_of_dirichlet_boundaries:
            ### 2D boundaries
            if specs['type'] == 'line':
                self.boundaries.append( bdry_line( point = specs['point'], 
                                                   normal = specs['normal'],
                                                   endpoints = specs['endpoints'],
                                                   boundary_condition = specs['boundary_condition'] ))
        # Unpack any periodic boundaries
        self.periodic_boundaries = []
        for specs in list_of_periodic_boundaries:
            self.periodic_boundaries.append( bdry_periodic( variable = specs['variable'],
                                                            base = specs['base'],
                                                            top = specs['top']  ))
class bdry_line:
    ### Class structure for a line boundary
    ###       normal vector points inside
    
    def __init__(self, point, normal, endpoints, boundary_condition):
        self.point = torch.tensor(  point )
        self.normal = torch.tensor( normal )
        self.constant = -sum( self.normal*self.point )
        
        self.bdry_cond = boundary_condition
        
        self.endpoints = torch.tensor(endpoints)
        
    def make_bdry_pts(self, num_bdry, boundingbox, is_unsteady, other_bdrys):
        
        if is_unsteady:
            Xbdry = torch.cat( ( (self.endpoints[1] - self.endpoints[0] )*torch.rand((num_bdry,1)) + self.endpoints[0],
                                 (boundingbox[-1][1] - boundingbox[-1][0])*torch.rand((num_bdry,1)) + boundingbox[-1][0]), dim=1)
        else:    
            Xbdry = ( self.endpoints[1] - self.endpoints[0] )*torch.rand((num_bdry,1)) + self.endpoints[0]
 
        ### Check if outside other bdrys
        ### and remake bdry points
        outside = torch.zeros(Xbdry.size(0), dtype=torch.bool)

        for bdry in other_bdrys:
            outside += bdry.dist_to_bdry(Xbdry) < 0
        
        if any(outside):
            Xbdry[outside,:] = self.make_bdry_pts(torch.sum(outside), boundingbox, is_unsteady, other_bdrys)

        return Xbdry
    
    def dist_to_bdry(self, X):
        ### Signed distance to boundary
        ### positive = inside domain
        ### negative = outside domain
        distance = torch.sum( self.normal.to(X.device)*X[:,:2], dim=1) + self.constant
        
        return distance
    
    def plot_bdry(self, num_bdry):
        Xplot = ( self.endpoints[1] - self.endpoints[0] )*torch.linspace(0, 1, num_bdry)[:,None] + self.endpoints[0]
        
        return Xplot

def generate_interior_points( boundingbox, boundaries):
    ### Generate points inside the domain
    
    ### Uniform Grid
    #x1 = (boundingbox[0][1] - boundingbox[0][0])*torch.linspace( dt, 1-dt, numpts_x ) + boundingbox[0][0]
    #x2 = (boundingbox[1][1] - boundingbox[1][0])*torch.linspace( dt, 1-dt, numpts_y ) + boundingbox[1][0]
    #X = torch.cartesian_prod( x1, x2)

    ### Randomly
    x1 = (boundingbox[0][1] - boundingbox[0][0])*torch.rand(numpts) + boundingbox[0][0]
    x2 = (boundingbox[1][1] - boundingbox[1][0])*torch.rand(numpts) + boundingbox[1][0]
    X = torch.stack([x1, x2], dim=1)

    outside = torch.zeros( X.size(0), dtype=torch.bool)
    for bdry in boundaries:
        outside += bdry.dist_to_bdry(X) < 0
    
    if any(outside):
        X[outside,:] = generate_interior_points(torch.sum(outside), boundingbox, boundaries)
        
    return X

class Walker_Data(torch.utils.data.Dataset):
    
    def __init__(self, boundingbox, boundaries):
        
        ### Define bdry exit condition
        def find_bdry_exit(Xold, Xnew, bdry, tol):
            ### Bisection algorithm to find the exit between Xnew and Xold up to a tolerance 
            
            Xmid = (Xnew + Xold)/2
            
            dist = bdry.dist_to_bdry(Xmid)
            
            # above tolerance = inside
            # below tolerance = outside
            above_tol = dist > tol
            below_tol = dist < -tol
            
            if torch.sum(above_tol + below_tol) > 0:
                Xnew[below_tol,:] = Xmid[below_tol,:]
                Xold[above_tol,:] = Xmid[above_tol,:]
                
                Xmid[above_tol + below_tol,:] = find_bdry_exit(Xold[above_tol + below_tol,:], Xnew[above_tol + below_tol,:], bdry, tol)

            return Xmid

        ### Generate points -- requires grad
        self.Xold = generate_interior_points(boundingbox, boundaries).requires_grad_(True)

        ### Move Walkers
        Uold = model_velocity(self.Xold)
        self.Xold.requires_grad_(False)

        Zt = np.sqrt(dt)*torch.randn((numpts*num_ghost, x_dim))

        self.Xnew = self.Xold.detach().repeat(num_ghost,1) - dt*Uold.detach().repeat(num_ghost,1) + np.sqrt(2*mu)*Zt

        for bdry in Domain.boundaries:
            outside_bdry = bdry.dist_to_bdry(self.Xnew) < 0
            
            self.Xnew[outside_bdry,:] = find_bdry_exit(self.Xold.detach().repeat(num_ghost,1)[outside_bdry,:], self.Xnew[outside_bdry,:], bdry, tol)
        
        self.Xnew.requires_grad=True
        Unew = model_velocity(self.Xnew).reshape(num_ghost, numpts, output_dim).mean(0)
        
        ###
        self.target = (Unew - Uold).detach()/dt
        ### Truncate?
        #self.target = torch.minimum(self.target, )
        self.Xnew = self.Xnew.detach().reshape(num_ghost,numpts,x_dim)

    def __len__(self):
        ### How many data points are there?
        return numpts
    
    def __getitem__(self, index):
        ### Gets one sample of data
        ### 
        return self.Xold[index,:], self.Xnew[:,index,:], self.target[index,:]

### Functions
def eval_gradient(X, model):
    X.requires_grad = True
    a = model(X)

    grad_model = torch.autograd.grad(a, X, grad_outputs = torch.ones_like(a), create_graph = True, 
                                       retain_graph = True, only_inputs = True)[0]

    return grad_model

In [7]:
nn_param = {'depth': nn_depth,
            'width': nn_width,
            'x_dim': x_dim,
            'is_unsteady': is_unsteady ,
            'output_dim': output_dim
            }

eval_model_param={'dt': dt,
                  'forcing': forcing}

### Make boundaries defining the domain
Domain = Domain(is_unsteady, boundingbox, list_of_dirichlet_boundaries, list_of_periodic_boundaries)

model_velocity = torch.load("savedmodels/JCPexample6.pt").to('cpu').eval()

### Initialize the Model
MyNeuralNetwork = PotentialNN
model_pressure = MyNeuralNetwork(**nn_param).to(dev)

mseloss = nn.MSELoss(reduction = 'mean')
optimizer = optim.Adam(model_pressure.parameters(), 
                        lr=learning_rate, 
                        betas=adam_beta, 
                        weight_decay=weight_decay)

### Create Walkers and Boundary points and Organize into DataLoader
RWalkers = Walker_Data(boundingbox, Domain.boundaries)
RWalkers_batch = torch.utils.data.DataLoader(RWalkers, batch_size=num_batch, shuffle=True)

if update_walkers == 'move':
    move_RWalkers = torch.zeros_like(RWalkers.location)

###
x_fix = torch.tensor(fix_pressure_at, dtype=torch.float, device=dev)
p_fix = fixed_pressure*torch.ones((1), device=dev)

In [8]:
start_time = time.time()

for step in range(num_epoch):

    # Try fixing one point to have 0 pressure
    #loss = lambda_fix*mseloss( model_pressure(x_fix), p_fix)
    #loss.backward()

    # Random Walkers - do in batches
    for Xold, Xnew, target in RWalkers_batch:

        # Send to GPU and set requires grad flag
        Xold = Xold.to(dev).requires_grad_(True)
        Xnew = Xnew.reshape(num_batch*num_ghost,x_dim).to(dev).requires_grad_(True)
        target = target.detach().to(dev)

        # No backprop wrt Xnew
        grad_pressure = model_pressure(Xold)
        #grad_pressure = eval_gradient(Xold, model_pressure)
        #grad_pressure = (eval_gradient(Xold, model_pressure) + eval_gradient(Xnew, model_pressure).reshape(num_ghost, num_batch, output_dim).mean(0).detach() )/2

        # Calculate loss
        loss = lambda_bell*mseloss(grad_pressure, target)
        loss.backward()

    optimizer.step()
    optimizer.zero_grad()

    if (step+1) % update_walkers_every == 0:
        if update_walkers == 'remake':
            RWalkers = DRLPDE_functions.DefineDomain.Walker_Data(boundingbox, Domain.boundaries)
            RWalkers_Batch = torch.utils.data.DataLoader(RWalkers, batch_size=num_batch, shuffle=True)


    if step == 0:
        print('No errors in first epoch: Training will continue')
    if (step+1) % update_print_every == 0:
        current_time = time.time()
        np.set_printoptions(precision=2)
        print('step = {0}/{1}, {2:2.3f} s/step, time-to-go:{3:2.0f}s'.format(
                step+1, num_epoch, (current_time - start_time) / (step + 1), 
            (current_time - start_time) / (step + 1) * (num_epoch - step - 1)))
    

No errors in first epoch: Training will continue
step = 1000/5000, 0.153 s/step, time-to-go:613s
step = 2000/5000, 0.149 s/step, time-to-go:446s
step = 3000/5000, 0.147 s/step, time-to-go:294s
step = 4000/5000, 0.146 s/step, time-to-go:146s
step = 5000/5000, 0.146 s/step, time-to-go: 0s


In [9]:
torch.save(model_pressure, "savedmodels/" + 'pressure_test'+ ".pt")